In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import gym
from gym import spaces
import pygame
import sys
import time
import Wumpus_Env
class QLearningAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.5  # Exploration-exploitation trade-off
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self.build_model()

    def build_model(self):
        model = tf.keras.Sequential([
            layers.Dense(24, input_shape=(self.state_size,), activation='relu'),
            layers.Dense(24, activation='relu'),
            layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=self.learning_rate), loss='mse')
        return model

    def act(self, state, action_size):
        self.action_size = action_size
        print(self.action_size)
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def train(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = (reward + self.gamma * np.max(self.model.predict(next_state)[0]))
        target_f = self.model.predict(state)
        target_f[0][action] = target
        self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


def preprocess_state(state):
    # Preprocess the state if needed
    return np.reshape(state, [1, len(state)])


def main():
    env = Wumpus_Env.WumpusWorldEnv(size=4, number_of_pits=3)
    state_size = len(env.reset())
    action_size = env.action_space.n

    agent = QLearningAgent(state_size, action_size)

    episodes = 1000
    for episode in range(episodes):
        state = preprocess_state(env.reset())

        total_reward = 0
        done = False

        while not done:
            action = agent.act(state, env.action_space.n)
            next_state, reward, done, _ = env.step(action)
            next_state = preprocess_state(next_state)
            agent.train(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            
            env.render()
            print(action)

        print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {agent.epsilon}")

    # Evaluate the trained agent
    state = preprocess_state(env.reset())
    done = False
    while not done:
        action = agent.act(state)
        next_state, _, done, _ = env.step(action)
        state = preprocess_state(next_state)
        env.render()

    pygame.quit()


if __name__ == "__main__":
    main()


(3, 1)
(3, 1)
8
(3, 1)
(3, 1)
1/1 [==============================] - 0s 27ms/step
2
8
(3, 1)
(3, 1)
1/1 [==============================] - 0s 24ms/step
2
Episode: 1, Total Reward: 29, Epsilon: 1.4850375
(3, 1)
8
(3, 1)
(3, 1)
1/1 [==============================] - 0s 25ms/step
7
4
(3, 1)
1/1 [==============================] - 0s 26ms/step
3
4
(3, 1)
1/1 [==============================] - 0s 23ms/step
3
4
(3, 1)
1/1 [==============================] - 0s 24ms/step
0
4
(3, 1)
1/1 [==============================] - 0s 23ms/step
0
4
(3, 1)
(3, 1)
1/1 [==============================] - 0s 23ms/step
2
4
(3, 1)
1/1 [==============================] - 0s 23ms/step
0
4
(3, 1)
(3, 1)
1/1 [==============================] - 0s 22ms/step
3
4
(3, 1)
(3, 1)
1/1 [==============================] - 0s 23ms/step
1
Episode: 2, Total Reward: -38, Epsilon: 1.4195318697201647
(3, 1)
8
(3, 1)
(3, 1)
1/1 [==============================] - 0s 22ms/step
5
4
(3, 1)
1/1 [==============================] - 0s 21ms/ste

KeyboardInterrupt: 